<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_HR_gr1/blob/main/Kiparenko%20Nikolai/INSPECTRUM_CLINIC_HR_RECSYS_GR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Работа с массивами данных
import numpy as np 

# Работа с таблицами
import pandas as pd

# Функции-утилиты для работы с категориальными данными
from tensorflow.keras import utils

# Класс для конструирования последовательной модели нейронной сети
from tensorflow.keras.models import Sequential

# Основные слои
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Embedding, Flatten, Activation 

# Токенизатор для преобразование текстов в последовательности
from tensorflow.keras.preprocessing.text import Tokenizer

# Заполнение последовательностей до определенной длины
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Матрица ошибок классификатора
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Кодирование тестовых меток
from sklearn.preprocessing import LabelEncoder

# Разбиение на тренировочную и тестовую выборки
from sklearn.model_selection import train_test_split

# Загрузка датасетов из облака google
import gdown

# Функции операционной системы
import os

# Работа со временем
import time

# Регулярные выражения
import re

# Запись в файлы и чтение из файлов структур данных Python
import pickle

# Отрисовка графиков
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import gdown                                      # Подключим функцию gdown
gdown.download('https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_HR/datasets/%D0%94%D0%B0%D1%82%D0%B0%D0%A1%D0%B5%D1%82%20%D0%9E%D1%86%D0%B5%D0%BD%D0%BA%D0%B0%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5%20581%20%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5.xlsx','dataS.xlsx', quiet=True)   

'dataS.xlsx'

In [33]:
data = pd.read_excel (r'dataS.xlsx', index_col=0)

data.columns = ['Response_date', 'Full_name', 'Gender', 'Salary', 'Job', 'City', 'Employment', 'Schedule', 'Experience', 'Last_place', 'Position', 'Education', 'Update_resume', 'Suitable']

In [34]:
data.head(10)

,Response_date,Full_name,Gender,Salary,Job,City,Employment,Schedule,Experience,Last_place,Position,Education,Update_resume,Suitable
0,2023-02-23 00:00:00,Bessonova Mariya,Женщина,90 000 руб.,Growth Product Manager,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 10 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2023, Нетоло́гия, 2014, Университет Ро...",23 февраля 2023 в 16:30,да
1,2023-01-11 00:00:00,Ефремова Екатерина Вячеславовна,Женщина,не указана,Growth Product Manager,"Москва, Готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —11 лет 11 месяцев,Банк Открытие,Product owner трайб сбережения и инвестиции,"Высшее, 2022, GeekBrains, 2010, Московский худ...",8 февраля 2023 в 09:15,да
2,2023-02-27 00:00:00,Фролов Артур Михайлович,"Мужчина, 33 года, родился 23 августа 1989",150 000 руб.,Growth Product Manager,"Санкт-Петербург, Готов к переезду: Москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —7 лет 2 месяца,DIKIY LLC,Директор по маркетингу и бренд-менеджер,"Высшее, 2013, Санкт-Петербургский государствен...",28 февраля 2023 в 09:41,да
3,2023-01-11 00:00:00,Хозаев Расул Шагаманович,"Мужчина, 32 года, родился 7 декабря 1990",не указана,Growth Product Manager,"Москва, м. Молодежная, Хочу переехать, готов к...","стажировка, частичная занятость, полная занятость","удаленная работа, гибкий график, полный день",Опыт работы —8 лет 10 месяцев,WeInvest,Product Manager (team leader),"Магистр, 2014, Московский государственный техн...",24 января 2023 в 12:07,да
4,2023-01-11 00:00:00,Шадрин Олег Михайлович,"Мужчина, 26 лет, родился 21 февраля 1996",150 000 руб.,Growth Product Manager,"Пермь, Хочу переехать, готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —5 лет 9 месяцев,Индивидуальное предпринимательство / частная п...,Product/Project manager,"Высшее, 2021, Российская академия народного хо...",6 февраля 2023 в 11:18,да
5,2023-02-25 00:00:00,Bessonova Mariya,"Женщина, 31 год, родилась 26 июня 1991",120 000 руб.,Product manager в Growth Hacking команду,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 11 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2014, Университет Российской академии ...",23 февраля 2023 в 16:46,да
6,2023-02-26 00:00:00,Ермакова Лилия,"Женщина, 36 лет, родилась 20 марта 1987",100 000 руб.,Директор медицинской клиники,"Уфа, Не готова к переезду, готова к редким ком...","полная занятость, частичная занятость, проектн...","полный день, гибкий график",Опыт работы —13 лет 9 месяцев,"ОАО ""Фармстандарт-УфаВИТА""",Начальник участка инъекционных препаратов,"Высшее, 2009, Башкирский государственный аграр...",9 марта 2023 в 22:27,да
7,2023-01-26 00:00:00,Маннанова Ксения Дмитриевна,"Женщина, 24 года, родилась 27 апреля 1998",30 000 руб.,HR-специалист,"Уфа, Не готова к переезду, не готова к команди...",полная занятость,полный день,Опыт работы —5 лет,Tiqum,Контент-менеджер,"Среднее специальное, 2018, УТЭК",15 февраля 2023 в 10:38,да
8,2023-02-28 00:00:00,Ахметьянова Кристина,"Женщина, 31 год, родилась 16 июля 1991",40 000 руб.,HR-специалист,"Уфа, Не готова к переезду, не готова к команди...",полная занятость,полный день,Опыт работы —10 лет 3 месяца,МБОУ СОШ,Учитель русского языка и литературы,"Магистр, 2015, Башкирский государственный педа...",26 марта 2023 в 19:29,да
9,2023-02-20 00:00:00,Иванова Дарья Дмитриевна,"Женщина, 28 лет, родилась 5 декабря 1994",не указана,HR-специалист,"Уфа, Не готова к переезду, не готова к команди...",полная занятость,полный день,Опыт работы —4 года 4 месяца,"Газпромбанк, ОАО",Специалист,"Среднее специальное, 2014, Уфимский Политехнич...",1 марта 2023 в 13:26,да


In [35]:
Birth = data['Gender'].apply(lambda x: x.split(',')[2].strip() if len(x.split(','))>2 else '')
data = data.assign(Birth=Birth)

Age = data['Gender'].apply(lambda x: x.split(',')[1].strip() if len(x.split(','))>1 else '')
data = data.assign(Age=Age)


In [41]:
for i, row in data.iterrows(): 
  row.Gender = row.Gender.split(',')[0].strip()   

  row.Birth = row.Birth.replace('родился','')
  row.Birth = row.Birth.replace('родилась','')
  
  row.Age = row.Age.replace('года','')
  row.Age = row.Age.replace('год','')
  row.Age = row.Age.replace('лет','')
  
  row.Salary = row.Salary.replace(' ','') 
  row.Salary = int(row.Salary.replace('KZT',''))*0.18  if row.Salary.find('KZT') != -1 else row.Salary
  row.Salary = int(row.Salary.replace('USD',''))*78  if row.Salary.find('USD') != -1 else row.Salary
  row.Salary = int(row.Salary.replace('EUR',''))*85  if row.Salary.find('EUR') != -1 else row.Salary
  row.Salary = row.Salary.replace('руб.','')

In [42]:
data.head(10)

,Response_date,Full_name,Gender,Salary,Job,City,Employment,Schedule,Experience,Last_place,Position,Education,Update_resume,Suitable,Birth,Age
0,2023-02-23 00:00:00,Bessonova Mariya,Женщина,90000руб.,Growth Product Manager,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 10 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2023, Нетоло́гия, 2014, Университет Ро...",23 февраля 2023 в 16:30,да,,
1,2023-01-11 00:00:00,Ефремова Екатерина Вячеславовна,Женщина,неуказана,Growth Product Manager,"Москва, Готова к переезду, готова к командировкам",полная занятость,"удаленная работа, гибкий график, полный день",Опыт работы —11 лет 11 месяцев,Банк Открытие,Product owner трайб сбережения и инвестиции,"Высшее, 2022, GeekBrains, 2010, Московский худ...",8 февраля 2023 в 09:15,да,,
2,2023-02-27 00:00:00,Фролов Артур Михайлович,Мужчина,150000руб.,Growth Product Manager,"Санкт-Петербург, Готов к переезду: Москва, гот...","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —7 лет 2 месяца,DIKIY LLC,Директор по маркетингу и бренд-менеджер,"Высшее, 2013, Санкт-Петербургский государствен...",28 февраля 2023 в 09:41,да,23 августа 1989,33
3,2023-01-11 00:00:00,Хозаев Расул Шагаманович,Мужчина,неуказана,Growth Product Manager,"Москва, м. Молодежная, Хочу переехать, готов к...","стажировка, частичная занятость, полная занятость","удаленная работа, гибкий график, полный день",Опыт работы —8 лет 10 месяцев,WeInvest,Product Manager (team leader),"Магистр, 2014, Московский государственный техн...",24 января 2023 в 12:07,да,7 декабря 1990,32
4,2023-01-11 00:00:00,Шадрин Олег Михайлович,Мужчина,150000руб.,Growth Product Manager,"Пермь, Хочу переехать, готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день",Опыт работы —5 лет 9 месяцев,Индивидуальное предпринимательство / частная п...,Product/Project manager,"Высшее, 2021, Российская академия народного хо...",6 февраля 2023 в 11:18,да,21 февраля 1996,26
5,2023-02-25 00:00:00,Bessonova Mariya,Женщина,120000руб.,Product manager в Growth Hacking команду,"Индонезия, Не готова к переезду, не готова к к...","проектная работа, частичная занятость, полная ...",удаленная работа,Опыт работы —5 лет 11 месяцев,PT MYBALI APP GROUP,Менеджер по продукту,"Высшее, 2014, Университет Российской академии ...",23 февраля 2023 в 16:46,да,26 июня 1991,31
6,2023-02-26 00:00:00,Ермакова Лилия,Женщина,100000руб.,Директор медицинской клиники,"Уфа, Не готова к переезду, готова к редким ком...","полная занятость, частичная занятость, проектн...","полный день, гибкий график",Опыт работы —13 лет 9 месяцев,"ОАО ""Фармстандарт-УфаВИТА""",Начальник участка инъекционных препаратов,"Высшее, 2009, Башкирский государственный аграр...",9 марта 2023 в 22:27,да,20 марта 1987,36
7,2023-01-26 00:00:00,Маннанова Ксения Дмитриевна,Женщина,30000руб.,HR-специалист,"Уфа, Не готова к переезду, не готова к команди...",полная занятость,полный день,Опыт работы —5 лет,Tiqum,Контент-менеджер,"Среднее специальное, 2018, УТЭК",15 февраля 2023 в 10:38,да,27 апреля 1998,24
8,2023-02-28 00:00:00,Ахметьянова Кристина,Женщина,40000руб.,HR-специалист,"Уфа, Не готова к переезду, не готова к команди...",полная занятость,полный день,Опыт работы —10 лет 3 месяца,МБОУ СОШ,Учитель русского языка и литературы,"Магистр, 2015, Башкирский государственный педа...",26 марта 2023 в 19:29,да,16 июля 1991,31
9,2023-02-20 00:00:00,Иванова Дарья Дмитриевна,Женщина,неуказана,HR-специалист,"Уфа, Не готова к переезду, не готова к команди...",полная занятость,полный день,Опыт работы —4 года 4 месяца,"Газпромбанк, ОАО",Специалист,"Среднее специальное, 2014, Уфимский Политехнич...",1 марта 2023 в 13:26,да,5 декабря 1994,28


In [ ]:
# Используется встроенный в Keras токенизатор для разбиения текста и построения частотного словаря
tokenizer = Tokenizer(num_words=5, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='неизвестное_слово', char_level=False)


# Построение частотного словаря по текстам
#tokenizer.fit_on_texts(sample_text)